In [1]:
import os

# Path to the dataset directory
dataset_path = '/kaggle/input/dataset9'

# List all files inside the dataset folder
files = os.listdir(dataset_path)
print("Files in dataset9:", files)


Files in dataset9: ['final_mapped_categories.csv']


In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)


✅ Using device: cuda


In [3]:
import pandas as pd

# Load the CSV file
df = pd.read_csv(f'{dataset_path}/final_mapped_categories.csv')

# Show first 5 rows
df.head()


,Recipe URL,Recipe name,Preparation time,Servings,List of ingredients,List of instructions,Category,Region/Cuisine,Calories_Per_Serving,Broad_Category
0,https://www.food.com/recipe/cheesy-grits-casse...,cheesy grits casserole,65.0,6.0,4cupswater 1teaspoonsalt 1cupquickcooking grit...,heat water and salt to boiling gradually add g...,oven,Unknown,493.0,Bread
1,https://www.food.com/recipe/crock-pot-beer-bra...,crock pot beer brats,440.0,4.0,8bratwursts 2tablespoonsolive oil 2garlic clov...,heat oil and garlic in a large skillet add bra...,pork,Unknown,493.0,Meat/Poultry
2,https://www.food.com/recipe/banana-bread-with-...,banana bread with coconut milk,40.0,10.0,4ripe mashedbananas 712ounces canscoconut milk...,mix wet ingredients and dry ingredients mixtur...,quick breads,Unknown,493.0,Bread
3,https://www.food.com/recipe/pasta-salad-suprem...,pasta salad supreme,25.0,6.0,16ouncesrotini pastaor 16 ouncesshell pastasom...,cook pasta rinse under cold water and drain we...,vegetable,Unknown,493.0,Vegetable
4,https://www.food.com/recipe/bread-machine-monk...,bread machine monkey pull aparts,70.0,11.0,112teaspoonsactive dry yeast 114cupsbread flou...,add ingredients to your bread maker according ...,breads,Unknown,493.0,Bread


In [24]:
import pandas as pd

df['text'] = df['Recipe name'].fillna('') + ' ' + \
             df['List of ingredients'].fillna('') + ' ' + \
             df['List of instructions'].fillna('')


In [25]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df['Category'].fillna('Unknown'))


In [26]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.to(device)  # 🚨 Important for GPU!
model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [27]:
from tqdm import tqdm
import numpy as np

def get_bert_embeddings_batch(texts, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True,
                           truncation=True, max_length=128).to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Move back to CPU
        embeddings.extend(cls_embeddings)
    return np.array(embeddings)




In [28]:
bert_features = get_bert_embeddings_batch(df['text'].tolist(), batch_size=32)

 57%|█████▋    | 896/1563 [04:01<02:59,  3.71it/s]


KeyboardInterrupt: 

In [ ]:
import joblib
joblib.dump(bert_features, "bert_features.pkl")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

# 🌟 Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✨ Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    bert_features, y, test_size=0.2, random_state=42
)

# 🌟 Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

# 🔢 Determine number of output classes
num_classes = len(set(y))  # e.g., 2 for binary

# 🔧 Define model (Linear = Logistic Regression)
model = nn.Sequential(
    nn.Linear(X_train.shape[1], num_classes)  # 768 → classes
).to(device)

# 🔥 Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🧠 Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")


In [ ]:
# 🎯 Evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted_labels = torch.argmax(predictions, axis=1)

accuracy = (predicted_labels == y_test_tensor).float().mean()
print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")


In [ ]:
# Track loss over epochs
losses = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")
        
# Plotting
import matplotlib.pyplot as plt

plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Time")
plt.grid(True)
plt.show()


In [ ]:
import joblib
joblib.dump(le, "/kaggle/working/label_encoder.pkl")


In [ ]:
# 🔐 Save model weights
torch.save(model.state_dict(), "/kaggle/working/mlp_model.pt")
print("✅ Model saved successfully.")


In [ ]:
import numpy as np

if torch.is_tensor(bert_features):
    bert_features = bert_features.cpu().numpy()


In [ ]:
!pip install transformers datasets --quiet


In [30]:
import pickle
from sklearn.preprocessing import LabelEncoder

# 👇 Load the file
with open('/kaggle/working/label_encoder.pkl', 'rb') as f:
    obj = pickle.load(f)

# 🧠 Check what was saved
if isinstance(obj, LabelEncoder):
    le = obj
    print("✅ Loaded full LabelEncoder")
elif isinstance(obj, (list, tuple, np.ndarray)):
    le = LabelEncoder()
    le.classes_ = obj
    print("⚠️ Loaded class array, recreated LabelEncoder")
else:
    raise ValueError("❌ Unknown label_encoder.pkl format!")

# ✅ Get number of classes for model
num_labels = len(le.classes_)
print("Number of labels:", num_labels)


⚠️ Loaded class array, recreated LabelEncoder
Number of labels: 591


In [35]:

# Combine text fields
df['text'] = df['Recipe name'].fillna('') + ' ' + \
             df['List of ingredients'].fillna('') + ' ' + \
             df['List of instructions'].fillna('')

# Apply loaded LabelEncoder
df['label'] = le.transform(df['Category'].fillna('Unknown'))


In [36]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

train_df, test_df = train_test_split(df[['text', 'label']], test_size=0.2, random_state=42)
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)


In [37]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_fn(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds = test_ds.map(tokenize_fn, batched=True)


Map:   0%|          | 0/40009 [00:00<?, ? examples/s]

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

In [38]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-39-9ab9ba8a0af5>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

In [ ]:
eval_results = trainer.evaluate()
print("📈 Evaluation Results:", eval_results)


In [ ]:
from sklearn.metrics import classification_report

preds = trainer.predict(test_ds)
pred_labels = preds.predictions.argmax(-1)

print(classification_report(test_df['label'], pred_labels, target_names=le.classes_))


In [ ]:
# 📦 Install required libraries
!pip install transformers datasets --quiet

# 🔽 Imports
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pickle



# 🧠 Combine text fields
df['text'] = df['Recipe name'].fillna('') + ' ' + \
             df['List of ingredients'].fillna('') + ' ' + \
             df['List of instructions'].fillna('')

# 🏷️ Encode target labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['Category'].fillna('Unknown'))
num_labels = len(le.classes_)

# ✂️ Train-test split
train_df, test_df = train_test_split(df[['text', 'label']], test_size=0.2, random_state=42)

# 🔄 Convert to HuggingFace Dataset
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

# ✏️ Tokenize
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds = test_ds.map(tokenize_fn, batched=True)

train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 🔧 Load BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

# ⚙️ Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Replaces deprecated evaluation_strategy
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    disable_tqdm=False
)

# 🧠 Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer
)

# 🚀 Train the model
trainer.train()

# 📊 Evaluate
eval_results = trainer.evaluate()
print("📈 Evaluation:", eval_results)

# 💾 Save model and label encoder
model.save_pretrained("/kaggle/working/bert_finetuned/")
tokenizer.save_pretrained("/kaggle/working/bert_finetuned/")
with open("/kaggle/working/label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/40009 [00:00<?, ? examples/s]

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-d810f935d5a3>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_scheduler
from tqdm import tqdm

# 📌 Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🔥 Using device:", device)

# 📁 Load dataset
df = pd.read_csv('/kaggle/input/dataset9/final_mapped_categories.csv')
df['text'] = df['Recipe name'].fillna('') + ' ' + df['List of ingredients'].fillna('') + ' ' + df['List of instructions'].fillna('')

# 🔠 Encode labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['Category'].fillna('Unknown'))


# 🚫 Remove labels with <2 samples
label_counts = df['label'].value_counts()
valid_labels = label_counts[label_counts > 1].index
df = df[df['label'].isin(valid_labels)]


# ✂️ Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42, stratify=df['label']
)

# 🧙‍♂️ Tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

# 📦 Dataset class
class RecipeDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = RecipeDataset(train_encodings, train_labels)
val_dataset = RecipeDataset(val_encodings, val_labels)

# 📚 Dataloaders
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=16)

# 🧠 Load BERT with classification head
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))
model.to(device)

# ⚙️ Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
num_training_steps = len(train_loader) * 4  # 4 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 🚀 Training loop
EPOCHS = 4
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    print(f"📉 Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")

    # ✅ Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=1)
            correct += (predictions == batch['labels']).sum().item()
            total += batch['labels'].size(0)

    print(f"✅ Epoch {epoch+1} Validation Accuracy: {100 * correct / total:.2f}%")

print("🎉 Training complete! Krishna-Radha blessings on your BERT model 🦚✨")


🔥 Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

📉 Epoch 2 Loss: 1.9071
✅ Epoch 2 Validation Accuracy: 53.42%


Epoch 3 Training: 100%|██████████| 2491/2491 [15:22<00:00,  2.70it/s]


📉 Epoch 3 Loss: 1.6241
✅ Epoch 3 Validation Accuracy: 54.99%


Epoch 4 Training: 100%|██████████| 2491/2491 [15:22<00:00,  2.70it/s]


📉 Epoch 4 Loss: 1.4666
✅ Epoch 4 Validation Accuracy: 55.97%
🎉 Training complete! Krishna-Radha blessings on your BERT model 🦚✨


In [3]:
# 💾 Save the model
model.save_pretrained("bert_recipe_classifier")
tokenizer.save_pretrained("bert_recipe_classifier")


('bert_recipe_classifier/tokenizer_config.json',
 'bert_recipe_classifier/special_tokens_map.json',
 'bert_recipe_classifier/vocab.txt',
 'bert_recipe_classifier/added_tokens.json')

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification

# 🔁 Load your saved model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert_recipe_classifier")
model = BertForSequenceClassification.from_pretrained("bert_recipe_classifier")
model.to(device)  # ⚡ Send it to GPU again


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
EPOCHS = 9
start_epoch = 4  # Resume from here

for epoch in range(start_epoch, EPOCHS):
    model.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        # ✅ Track training accuracy
        correct_train += (predictions == batch['labels']).sum().item()
        total_train += batch['labels'].size(0)

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    train_acc = 100 * correct_train / total_train
    print(f"📉 Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")
    print(f"✅ Epoch {epoch+1} Training Accuracy: {train_acc:.2f}%")

    # 🧪 Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=1)
            correct += (predictions == batch['labels']).sum().item()
            total += batch['labels'].size(0)

    val_acc = 100 * correct / total
    print(f"🧪 Epoch {epoch+1} Validation Accuracy: {val_acc:.2f}%\n")





Epoch 5 Training: 100%|██████████| 2491/2491 [14:21<00:00,  2.89it/s]


📉 Epoch 5 Loss: 1.4134
✅ Epoch 5 Training Accuracy: 62.81%
🧪 Epoch 5 Validation Accuracy: 55.97%



Epoch 6 Training: 100%|██████████| 2491/2491 [14:20<00:00,  2.90it/s]


📉 Epoch 6 Loss: 1.4105
✅ Epoch 6 Training Accuracy: 63.07%
🧪 Epoch 6 Validation Accuracy: 55.97%



Epoch 7 Training: 100%|██████████| 2491/2491 [14:19<00:00,  2.90it/s]


📉 Epoch 7 Loss: 1.4112
✅ Epoch 7 Training Accuracy: 62.78%
🧪 Epoch 7 Validation Accuracy: 55.97%



Epoch 8 Training: 100%|██████████| 2491/2491 [14:19<00:00,  2.90it/s]


📉 Epoch 8 Loss: 1.4132
✅ Epoch 8 Training Accuracy: 63.08%
🧪 Epoch 8 Validation Accuracy: 55.97%



Epoch 9 Training: 100%|██████████| 2491/2491 [14:20<00:00,  2.90it/s]


📉 Epoch 9 Loss: 1.4123
✅ Epoch 9 Training Accuracy: 63.09%
🧪 Epoch 9 Validation Accuracy: 55.97%

